<a href="https://colab.research.google.com/github/SURIYASURENDHAR/internal-status/blob/main/Copy_of_external_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
# Load the dataset
df=pd.read_json("dataset.json")
df.head()

,externalStatus,internalStatus
0,PORT OUT,Port Out
1,TERMINAL IN,Inbound Terminal
2,PORT IN,Port In
3,Vessel departure from first POL (Vessel name :...,Departure
4,Vessel arrival at final POD (Vessel name : TIA...,Arrival


In [ ]:

 #Show data info
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   externalStatus  1222 non-null   object
 1   internalStatus  1222 non-null   object
dtypes: object(2)
memory usage: 19.2+ KB


In [ ]:
df.shape# how many columns and rows

(1222, 2)

In [ ]:
# Removing any duplicate rows
df.describe().T

,count,unique,top,freq
externalStatus,1222,108,Gate out,144
internalStatus,1222,15,Loaded on Vessel,331


In [ ]:
# Check for missing values
df.isnull().sum()


externalStatus    0
internalStatus    0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
# Data cleaning
df['externalStatus'] = df['externalStatus'].str.lower().str.strip()
df['internalStatus'] = df['internalStatus'].str.lower().str.strip()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Prepare text data
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['externalStatus'])
sequences = tokenizer.texts_to_sequences(df['externalStatus'])
padded = pad_sequences(sequences, maxlen=20, padding='post', truncating='post')

In [ ]:
# Prepare labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['internalStatus'])

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(padded, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
#Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16, input_length=20),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            80000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 15)                375       
                                                                 
Total params: 80783 (315.56 KB)
Trainable params: 80783 (315.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32,validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/30
31/31 [==============================] - 0s 5ms/step - loss: 0.0171 - accuracy: 0.9990 - val_loss: 0.0361 - val_accuracy: 0.9918
Epoch 2/30
31/31 [==============================] - 0s 3ms/step - loss: 0.0166 - accuracy: 0.9990 - val_loss: 0.0352 - val_accuracy: 0.9918
Epoch 3/30
31/31 [==============================] - 0s 3ms/step - loss: 0.0162 - accuracy: 0.9990 - val_loss: 0.0346 - val_accuracy: 0.9918
Epoch 4/30
31/31 [==============================] - 0s 4ms/step - loss: 0.0158 - accuracy: 0.9990 - val_loss: 0.0337 - val_accuracy: 0.9918
Epoch 5/30
31/31 [==============================] - 0s 3ms/step - loss: 0.0153 - accuracy: 0.9990 - val_loss: 0.0347 - val_accuracy: 0.9918
Epoch 6/30
31/31 [==============================] - 0s 3ms/step - loss: 0.0148 - accuracy: 0.9990 - val_loss: 0.0330 - val_accuracy: 0.9918
Epoch 7/30
31/31 [==============================] - 0s 4ms/step - loss: 0.0143 - accuracy: 0.9990 - val_loss: 0.0321 - val_accuracy: 0.9918
Epoch 8/30
31/31 [==